## Check HW








In [1]:

!lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         39 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  8
  On-line CPU(s) list:   0-7
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz
    CPU family:          6
    Model:               142
    Thread(s) per core:  2
    Core(s) per socket:  4
    Socket(s):           1
    Stepping:            12
    CPU(s) scaling MHz:  63%
    CPU max MHz:         4900.0000
    CPU min MHz:         400.0000
    BogoMIPS:            4599.93
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mc
                         a cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss 
                         ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc art
                          arch_perfmon pebs bts rep_good nopl xtopology nonstop_
                         tsc cpuid aperfmperf pni

In [2]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            15Gi       2.4Gi       9.0Gi       386Mi       3.9Gi        12Gi
Swap:          8.0Gi          0B       8.0Gi


In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Thu_Nov_18_09:45:30_PST_2021
Cuda compilation tools, release 11.5, V11.5.119
Build cuda_11.5.r11.5/compiler.30672275_0


In [2]:
!nvidia-smi

Tue Oct 18 15:12:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.76       Driver Version: 515.76       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:2D:00.0 Off |                  N/A |
| N/A   43C    P8    N/A /  N/A |      1MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install the environment

In [8]:
!#pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

Defaulting to user installation because normal site-packages is not writeable
  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-04_h0z6g
  Running command git clone --filter=blob:none --quiet git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-04_h0z6g


In [ ]:
!rm -rf master.zip* nvcc4jupyter-master*
!wget https://github.com/andreinechaev/nvcc4jupyter/archive/refs/heads/master.zip
!unzip master.zip
!cd nvcc4jupyter-master; ls; python setup.py install

--2022-09-23 08:31:01--  https://github.com/andreinechaev/nvcc4jupyter/archive/refs/heads/master.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/andreinechaev/nvcc4jupyter/zip/refs/heads/master [following]
--2022-09-23 08:31:01--  https://codeload.github.com/andreinechaev/nvcc4jupyter/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 20.205.243.165
Connecting to codeload.github.com (codeload.github.com)|20.205.243.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5201 (5.1K) [application/zip]
Saving to: ‘master.zip’

master.zip          100%[===================>]   5.08K  --.-KB/s    in 0s      

2022-09-23 08:31:02 (58.3 MB/s) - ‘master.zip’ saved [5201/5201]

Archive:  master.zip
aac710a35f52bb78ab34d2e52517237941399eff
   creating: nvcc4jupyter-master/
 extracting

In [3]:
%load_ext nvcc_plugin

created output directory at /home/david/projects/uam/uam-high-perf-comp/practica2/material/src
Out bin /home/david/projects/uam/uam-high-perf-comp/practica2/material/result.out


## Run a program (using Google overpowered GPUs)



### Simple mode, everything on the same file

This is the only part of the notebook that really uses the extension nvcc_plugin

In [4]:
%%cu

#include <stdio.h>
#include <cuda.h>
#include <cuda_runtime.h>

__global__ void mykernel(void) {

}

int main(void) {
		mykernel<<<1,1>>>();
		printf("Hello world\n");
		return 0;
}

In file included from /usr/local/cuda/bin/../targets/x86_64-linux/include/cuda_runtime.h:83,
                 from <command-line>:
/usr/local/cuda/bin/../targets/x86_64-linux/include/crt/host_config.h:139:2: error: #error -- unsupported GNU version! gcc versions later than 11 are not supported! The nvcc flag '-allow-unsupported-compiler' can be used to override this version check; however, using an unsupported host compiler may cause compilation failure or incorrect run time execution. Use at your own risk.
  139 | #error -- unsupported GNU version! gcc versions later than 11 are not supported! The nvcc flag '-allow-unsupported-compiler' can be used to override this version check; however, using an unsupported host compiler may cause compilation failure or incorrect run time execution. Use at your own risk.
      |  ^~~~~



In [5]:
%%cu
#include <cuda.h>
#include <cuda_runtime.h>
#include <stdio.h>

__global__ void add(int *a, int *b, int *c) {
    *c = *a + *b;
}

int main(void) {
    int a, b, c;				    // host copies of a, b, c
	  int *d_a, *d_b, *d_c;		// device copies of a, b, c
	  int size = sizeof(int);

	  // Allocate space for device copies of a, b, c
	  cudaMalloc((void **)&d_a, size);
	  cudaMalloc((void **)&d_b, size);
	  cudaMalloc((void **)&d_c, size);

	  // Setup input values
	  a = 2;
	  b = 7;

	  // Copy inputs to device
	  cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
	  cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);

	  // Launch add() kernel on GPU
	  add<<<1,1>>>(d_a, d_b, d_c);

	  // Copy result back to host
	  cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);

	  // Show result
	  printf("%d + %d = %d", a, b, c);

	  // Cleanup
	  cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);
	  return 0;
}

In file included from /usr/local/cuda/bin/../targets/x86_64-linux/include/cuda_runtime.h:83,
                 from <command-line>:
/usr/local/cuda/bin/../targets/x86_64-linux/include/crt/host_config.h:139:2: error: #error -- unsupported GNU version! gcc versions later than 11 are not supported! The nvcc flag '-allow-unsupported-compiler' can be used to override this version check; however, using an unsupported host compiler may cause compilation failure or incorrect run time execution. Use at your own risk.
  139 | #error -- unsupported GNU version! gcc versions later than 11 are not supported! The nvcc flag '-allow-unsupported-compiler' can be used to override this version check; however, using an unsupported host compiler may cause compilation failure or incorrect run time execution. Use at your own risk.
      |  ^~~~~



### Running code in multiple source files

In [ ]:
!mkdir cuda
!rm -rf cuda/*

mkdir: cannot create directory ‘cuda’: File exists


In [ ]:
%%writefile cuda/kernel.cu
#include <cuda.h>

__global__ void add(int *a, int *b, int *c) {
    *c = *a + *b;
}

Writing cuda/kernel.cu


In [ ]:
%%writefile cuda/main.cu
#include <cuda.h>
#include <cuda_runtime.h>
#include <stdio.h>

__global__ void add(int *a, int *b, int *c);

int main(void) {
	int a, b, c;				    // host copies of a, b, c
	int *d_a, *d_b, *d_c;		// device copies of a, b, c
	int size = sizeof(int);

	// Allocate space for device copies of a, b, c
	cudaMalloc((void **)&d_a, size);
	cudaMalloc((void **)&d_b, size);
	cudaMalloc((void **)&d_c, size);

	// Setup input values
	a = 2;
	b = 7;

	// Copy inputs to device
	cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
	cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);

	// Launch add() kernel on GPU
	add<<<1,1>>>(d_a, d_b, d_c);

	// Copy result back to host
	cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);

	// Show result
	printf("%d + %d = %d", a, b, c);

	// Cleanup
	cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);
	return 0;
}

Writing cuda/main.cu


In [ ]:
!nvcc cuda/*.cu -o cuda/program
!./cuda/program
!rm -rf ./cuda/program

2 + 7 = 9

## Mount your Google Drive

In [ ]:
# Mount Google Drive
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/gdrive')

In [ ]:
# Copy source code to your folder
!cp /content/gdrive/MyDrive/XXX/* cuda/
# Create soft links
!ln -s /content/gdrive/MyDrive/XXXX cuda/XXXX
# List folder
!ls -la cuda/